# VTK 简介

Dash VTK的目标是将VTK/vtk.js可视化集成到Dash框架中。

[VTK](https://vtk.org/) 代表可视化工具包，用 C++ 编写的流行库，也可以在 Python 中使用，用于科学和医疗领域的数据处理和可视化。通常，VTK 用于可视化来自模拟或传感器（如 LIDAR 扫描仪）的 3D 几何图形。在医疗领域，VTK 通过进行体渲染和/或切片来呈现 3D 图像（即 CT、MRI 等）。

另一方面，[Vtk.js](https://kitware.github.io/vtk-js/) 是 VTK 的子集，专注于 JavaScript 世界中的渲染方面。Vtk.js 采用与其大哥 VTK/C++ 相同的架构和对象分解，但使其易于在浏览器内部使用。

Dash VTK 允许用户在服务器端使用 VTK 进行任何数据处理，并提供将可视化推送到客户端的基础设施，以获得更好的体验。Dash VTK 不需要 VTK，但可以无缝地利用它来查看点云、CFD 模拟或任何与 3D 网格或 3D 图像相关的内容。

在 VTK 中，有 3 种主要类型的对象，这些对象是理解其可视化原理的关键。首先是 `View`，它只是容器，用于显示您想要查看的任何 `DataSource` 的表现（`Representation`）。

## `View`

`View` 是 3D 视图，可以进行网格的几何渲染或 3D 图像的体渲染。通过配置，该视图可以作为 2D 视图使用，在使用平行投影时阻止旋转交互。`View` 组件可以通过以下一组属性进行配置。

```python
dash_vtk.View(
  id='vtk-view',
  background=[0, 0, 0],           # RGB 数组由介于 0 和 1 之间的浮点数值组成
  interactorSettings=[...],       # 将鼠标事件绑定到相机动作（旋转、平移、缩放等）
  cameraPosition=[x,y,z],         # 相机最初在 3D 世界中应放置的位置
  cameraViewUp=[dx, dy, dz],      # 作为初始相机视图向上的向量
  cameraParallelProjection=False, # 应该用透视图查看我们的 3D 作品，还是用没有深度感的平面视图
  triggerRender=0,                # 用于在时间戳不同时触发渲染的时间戳
  triggerResetCamera=0,           # 用于在时间戳不同时触发相机重置的时间戳
  # clickInfo,                    # 只读属性，用于检索拾取的表现（representation）id 和拾取(picking )信息
  # hoverInfo                     # 只读属性，用于检索拾取的表现（representation）id 和拾取(picking )信息
)
```

对于 `interactorSettings`，期望鼠标事件类型与动作相关联的列表。下面的示例是默认使用的设置：

```python
interactorSettings = [
  {
    button: 1,
    action: 'Rotate',
  }, {
    button: 2,
    action: 'Pan',
  }, {
    button: 3,
    action: 'Zoom',
    scrollEnabled: true,
  }, {
    button: 1,
    action: 'Pan',
    shift: true,
  }, {
    button: 1,
    action: 'Zoom',
    alt: true,
  }, {
    button: 1,
    action: 'ZoomToMouse',
    control: true,
  }, {
    button: 1,
    action: 'Roll',
    alt: true,
    shift: true,
  }
]
```

鼠标事件可以通过以下一组属性来标识：

- `button`: 1/2/3 # 应该按下哪个按钮
- `shift`: True/False # Shift键是否按下
- `alt`: True/False # Alt键是否按下
- `control`: True/False # Ctrl键是否按下
- `scrollEnabled`: True/False # 某些动作也可以通过滚动触发
- `dragEnabled`: True/False # 主要用于禁用默认的拖动行为

动作可以是以下之一：

- `Pan`: 将在垂直于相机的平面上平移对象
- `Zoom`: 根据拖动方向将对象拉近或拉远
- `Roll`: 将围绕视图方向旋转对象
- `ZoomToMouse`: 在保持最初在鼠标下的位置不变的同时进行缩放

## `Representation`

表现（Representation）负责将 `DataSource` 转换成可视化的内容，这些内容将在 `View` 中呈现。

到目前为止，在 `dash_vtk` 中提供了3种核心类型的 `Representation`：

- `GeometryRepresentation`：几何表现（`Representation`）将期望是网格，并将其渲染为几何渲染（想象三角形集合）。
- `VolumeRepresentation`：体积表现将期望 3D 图像，并使用体渲染技术进行渲染，该技术可以让您透视（雾状物体）。
- `SliceRepresentation`：切片表现将期望 3D 图像，并沿着给定轴对其进行切片。
应该将表现放在View的子元素中。

## `DataSource`

`DataSource` 可以是许多东西，但主要是能够产生数据的东西。换句话说，它可以是数据集或过滤器，用于处理某些数据并生成新的数据，甚至可以是读取器，用于读取某种输入（文件、URL等）并生成一些数据。任何 `DataSource` 都可以放在另一个 `DataSource` 的子元素中，后者将作为过滤器使用，或者简单地传递给 `Representation`。

在 `dash_vtk` 中，我们有几个对象属于这个类别。下面的列表为您提供了这些对象的概览。

- `Algorithm`：允许您实例化 vtk.js 算法，该算法可以是过滤器（如 `vtkWarpScalar`）或源（如 `vtkLineSource`, `vtkConeSource`, `vtkPlaneSource`, `vtkSphereSource`, `vtkCylinderSource`）。
- `ImageData`：到目前为止我们所说的3D图像。这个元素将让您定义组成3D图像的每一个部分。
- `PolyData`：一个表面网格（点、三角形等）。这个元素将让您定义网格的各个部分。
- `Reader`：与 `Algorithm` 类似，除了读取器有共同的 API，并且这个元素让您利用这些 API。
- `ShareDataSet`：允许您捕获任何 `DataSource`，并在另一个处理管道或表现中使其可用，而不会复制从服务器发送到客户端的数据。
- `Mesh`：类似于 `PolyData`，除了它有 Python 辅助函数，帮助您将 `vtkDataSet` 映射到 `Mesh` 的属性中。
- `Volume`：类似于 `ImageData`，除了它有 Python 辅助函数，帮助您将 `vtkImageData` 映射到 `Volume` 的属性中。